In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

In [ ]:
# Loading dotenv package for authorization
import os
from dotenv import load_dotenv

load_dotenv()
YELP_KEY = os.environ.get('YELP_key')

In [ ]:
# Ipython package for better readability
import requests
from IPython.display import JSON

In [ ]:
# inserting my authorization key
with open('YELP_KEY.txt') as content:
    yelp_key = content.read()

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Finding eateries close to 1000m of bike stations
import requests
API_KEY = os.environ.get('API_Key')
foursq_location = []
for cb_geo in range(len(df2_longitude)):
    
    url = f"https://api.foursquare.com/v3/places/search?query=restaurants&ll={df2_latitude[cb_geo]},{df2_longitude[cb_geo]}&radius=1000&fields=rating%2Cfsq_id%2Cname%2Ccategories%2Clocation%2Cdistance%2Cpopularity&sort=RELEVANCE"

    headers = {
        "accept": "application/json",
        "Authorization": API_KEY
               }

    response = requests.get(url, headers=headers)
    foursq_location.append(response.json())
JSON(foursq_location)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
name = []
rating = []
fsq_id = []
temp = {}
for i in  (range(len(foursq_location))):
    name.append(foursq_location[i]["results"][0]["name"])
    rating.append(foursq_location[i]["results"][0]["rating"])
    fsq_id.append(foursq_location[i]["results"][0]["fsq_id"])
    
temp = {"name":name,"rating": rating,"fsq_id":fsq_id}

In [ ]:
# Using JSON data and getting POI for every location
POI_list = df333
POI_data = []
for poi in POI_list:
    POI_data.append(poi)
df3333 = pd.json_normalize(poi)
df3333

Put your parsed results into a DataFrame

In [ ]:
# Normalizing data, droping some irrelevant columns and selecting few ones for futher analysis
df3 = pd.json_normalize(POI_data)
df3.drop(df3.iloc[:,7:] ,inplace=True, axis=1)
df3.drop(columns=['fsq_id','categories'],inplace=True, axis=1)
df3.columns=['distance','name','popularity','rating','address']
df3.to_csv('fourSQ.csv')
df3

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Finding eateries close to 1000m of bike stations
import requests
with open('YELP_KEY.txt') as content:
    yelp_key = content.read()
yelp_location = []
for cb_geo in range(len(df3_longitude)):
    
    url = f"https://api.yelp.com/v3/businesses/search?location=london&latitude={df3_latitude[cb_geo]}&longitude={df3_longitude[cb_geo]}&term=restaurants&radius=1000&categories=name%26categories%3Dlocation%26categories%3Ddistance%26categories%3Dpopularity%26categories%3Drating&sort_by=rating&limit=50"

    headers = {
        "accept": "application/json",
        "Authorization": yelp_key
               }

    response = requests.get(url, headers=headers)
    yelp_location.append(response.json())
JSON(yelp_location)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# POI for data through YELP
POI_list = yelp_location['businesses']
POI_data = []
for poi in POI_list:
    POI_data.append(poi)

Put your parsed results into a DataFrame

In [ ]:
# Normalizing data, droping some irrelevant columns and selecting few ones for further analysis
df4 = pd.json_normalize(POI_data)
df4.drop(columns=['url','alias','id','transactions','categories','image_url','is_closed','coordinates.latitude','coordinates.longitude','price','phone','display_phone'] ,inplace=True, axis=1)
df4.drop(df4.iloc[:,5:14] ,inplace=True, axis=1)
df4.columns=['name','review_count','rating','distance','location.address1']
df4.rename(columns = {'location.address1':'address','review_count':'review'},inplace=True)
df4

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided more information, simply for the fact that it provided me more parameters for the model. Certain feilds like reviews and ratings for all restaurants helped a lot. All can be seen from above queries.

Get the top 10 restaurants according to their rating

In [2]:
top10 = pd.read_csv('API_dataFinal.csv',index_col = 0)
top10.sort_values('rating')
top10.head(10)

,distance,name,popularity,rating,address
0,192,Hanoi Cafe,0.939773,8.1,98 Kingsland Rd
1,171,Viet Grill,0.973170,7.6,58 Kingsland Rd
2,255,Friends of Ours,0.972873,8.8,61 Pitfield St
3,230,Curious Yellow Kafe,0.961717,8.2,77 Pitfield St
4,251,Via Emilia,0.945041,8.4,37A Hoxton Square
5,241,Sông Quê,0.962018,8.0,134 Kingsland Rd
6,278,Homeslice Pizza,0.943472,8.7,374-378 Old St
7,283,The Clove Club,0.965496,8.8,380 Old St
8,291,The Breakfast Club,0.957459,8.1,2-4 Rufus St
9,311,MEAT Mission,0.974697,8.4,14-15 Hoxton Market
